In [24]:
# loading the annual report file
from langchain_community.document_loaders import PyPDFLoader

file_path = "Xchanging Solution Report 2025.pdf"
loader = PyPDFLoader(file_path=file_path)
docs = loader.load()
docs[0]

Document(metadata={'producer': 'Corel PDF Engine Version 23.1.0.389', 'creator': 'CorelDRAW 2021', 'creationdate': '2025-07-18T14:56:39+05:30', 'author': 'my', 'moddate': '2025-07-18T14:58:11+05:30', 'title': 'Xchanging Report Cover 2025.cdr', 'source': 'Xchanging Solution Report 2025.pdf', 'total_pages': 198, 'page': 0, 'page_label': '1'}, page_content='')

In [25]:
import pprint

pprint.pp(docs[0].metadata)

{'producer': 'Corel PDF Engine Version 23.1.0.389',
 'creator': 'CorelDRAW 2021',
 'creationdate': '2025-07-18T14:56:39+05:30',
 'author': 'my',
 'moddate': '2025-07-18T14:58:11+05:30',
 'title': 'Xchanging Report Cover 2025.cdr',
 'source': 'Xchanging Solution Report 2025.pdf',
 'total_pages': 198,
 'page': 0,
 'page_label': '1'}


In [26]:
pprint.pp(docs[3].page_content)

('3Xchanging Solutions LimitedAnnual Report 2024-25\n'
 'BOARD’S REPORT\n'
 'Dear Shareholders,\n'
 'The Board of Directors (“Board”) is pleased to present the Twenty\n'
 'Fourth Annual Report and the Audited Financial Statements of\n'
 'the Xchanging Solutions Limited (“ the Company ”) for the\n'
 'financial year ended March 31, 2025.\n'
 'A. FINANCIAL RESULTS\n'
 'The highlights of standalone and consolidated financial\n'
 'results of the Company for the financial year ended March\n'
 '31, 2025 and March 31, 2024 are as under:\n'
 '(Rs. in Lakhs)\n'
 'PARTICULARS STANDALONE CONSOLIDATED\n'
 'For the For the For the For the\n'
 'Financial Financial Financial Financial\n'
 'year year year year\n'
 'ended ended ended ended\n'
 'March 31, March 31, March 31, March 31,\n'
 '2025 2024 2025 2024\n'
 'Total Income 5,498 12,560 20,191 19,625\n'
 'Total Expenditure 3,139 2,825 13,850 13,007\n'
 'Profit before Finance Costs, 2392 9,799 7,006 6,859\n'
 'Depreciation, Tax and\n'
 'exceptional ite

In [28]:
# splitting the document
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_text = text_splitter.split_documents(docs)

In [29]:
print(len(splitted_text))
print(splitted_text[54].page_content)

832
Based on our verification of M/s. Xchanging Solutions Limited’s books, papers, minute books, forms and returns filed and other
records maintained by the Company and also the information provided by the Company, its officers and authorized representatives
during the conduct of secretarial audit, we hereby report that in our opinion, the Company has, during the audit period coverin g the
financial year ended on 31
st March 2025 commencing from April 1, 2024 to March 31, 2025, complied with the statutory provisions
listed hereunder and also that the Company has proper Board processes and compliance mechanism in place to the extent based
on the management representation letter/ confirmation received from the management, in the manner and subject to the reporting
made hereinafter. The members are requested to read this report along with our letter dated May 21, 2025, annexed to this repor t
as 
Annexure – A.


In [30]:
# embedding
from langchain_ollama.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

In [31]:
# vector store
from langchain_chroma import Chroma


vector_store = Chroma(
    collection_name="xchanging_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [32]:
vector_store.add_documents(splitted_text)

['56ea8625-351b-492f-a537-87b42e63f7a4',
 'aa01d761-740b-48f5-affc-a95fe9dfba53',
 '4a313a59-1a27-48d3-ba5a-7348ab5f96f9',
 '7bd5b4f5-d040-47a5-b121-227d1a8839c6',
 'b6f7b10d-9ed9-4bc5-8441-47078598c121',
 '67df7aa0-6d4d-403c-94e0-6514e0d29f02',
 '74c3159b-3826-4266-b7f7-c4c37e694394',
 '0ba47cde-73b9-46eb-bf65-d02c143633b0',
 '35b51b67-47d2-4c9b-be2a-7f2ddca08711',
 '8ec455a1-f78f-4702-b284-2dfe36d7efac',
 'a5e44b18-5ca1-4896-98fb-1557f9a67e83',
 'faefbb18-606a-41dd-a416-fef3982731ee',
 '67a8921b-f5f5-4de4-b8eb-f115b916a4c5',
 '0d0a19d9-9b52-43e6-8298-0fa61a6a25da',
 '9d04dd6d-cce2-4310-abb2-ffa1e6ea72da',
 '8d209fb6-b665-4efa-9d3e-ec385e7c4fc0',
 '1c4dea8c-2cfc-4b25-a760-0053a9ce129e',
 '5eb1b379-daaa-454a-b222-3fb828862c42',
 '8114fa93-3de6-4568-8a53-70c6d51de0f8',
 '7134452e-275f-4f38-81a5-786c9fb4dbec',
 'f46d2237-359e-4242-985b-79275d8dd5ee',
 '1547a41e-18a8-42f4-b8fa-57bff9c6e524',
 'eb84d4e1-f89c-4fdc-a74e-55587ccf93e9',
 '740b3f6c-07d0-4ae3-a5f8-95be5c3fa2b0',
 '77dc28a1-fd70-

In [33]:
results = vector_store.similarity_search(
    "when engaging in influencing public and regulatory policy, should do so in a manner that is responsible and transparent",
    k=1
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* PRINCIPLE 7 Businesses, when engaging in influencing public and regulatory policy, should do so in a manner that is
responsible and transparent.
Essential Indicators [{'total_pages': 198, 'page_label': '68', 'creationdate': '2025-07-18T14:56:39+05:30', 'moddate': '2025-07-18T14:58:11+05:30', 'page': 67, 'title': 'Xchanging Report Cover 2025.cdr', 'author': 'my', 'creator': 'CorelDRAW 2021', 'producer': 'Corel PDF Engine Version 23.1.0.389', 'source': 'Xchanging Solution Report 2025.pdf'}]


In [34]:
# integration with llm chatmodel
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")


In [ ]:
# prompt template
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    """
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [39]:
# retriever chain
from langchain.chains import create_retrieval_chain

retriever = vector_store.as_retriever()
retriever_chain = create_retrieval_chain(retriever, document_chain)

In [40]:
# get the response from the LLM 
response = retriever_chain.invoke({
    "input":"Create the summary of entire document in 10 lines as a Investor"
})

In [55]:
for i in range(4):
    print(response['context'][i].page_content)

The consolidated financial statements of the Group have been prepared based on a line-by-line consolidation of
Provide details of concentration of purchases and sales with trading houses, dealers, and related parties along-with loans and
advances & investments, with related parties in the following format:
statements may write to the Company. The financial
statement including the consolidated financial statement and
all other documents required to be attached with this report
=========================== =========================
EQUITY AND LIABILITIES
Equity
Equity share capital 14 11,140 11,140
Other equity 15 24,727 24,254
---------------------------- ----------------------------
Total equity 35,867 35,394
---------------------------- ----------------------------
Liabilities
Non-current liabilities
Financial liabilities
Lease liabilities 34 - 63
Borrowings 16 6,410 10,426
Deferred tax liabilities (net) 9 839 743
Provisions 17 113 143
---------------------------- ---------------------